In [6]:
%pip install spacy-transformers
%pip install gradio
%pip install haystack-ai accelerate sentence-transformers
%pip install 'sentence-transformers>=2.2.0'

  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
   ---------------------------------------- 0.0/343.2 kB ? eta -:--:--
   ------- -------------------------------- 61.4/343.2 kB 3.4 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/343.2 kB 3.4 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/343.2 kB 3.4 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/343.2 kB 3.4 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/343.2 kB 3.4 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/343.2 kB 3.4 MB/s eta 0:00:01
   ------- -------------------------------- 61.4/343.2 kB 3.4 MB/s eta 0:00:01
   ---------------------- --------------- 204.8/343.2 kB 541.9 kB/s eta 0:00:01
   --------------------------- ---------- 245.8/343.2 kB 580.1 kB/s eta 0:00:01
   -------------------------------------  337.9/343.2 kB 723.4 kB/s eta 0:00:01
   -------------------------------------- 343.2/343.2 kB 711.4 kB/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'sentence-transformers"


In [18]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 5.1 MB/s eta 0:00:03
     ---------------------------------------- 0.1/12.8 MB 2.1 MB/s eta 0:00:06
      --------------------------------------- 0.2/12.8 MB 1.6 MB/s eta 0:00:09
      --------------------------------------- 0.3/12.8 MB 1.7 MB/s eta 0:00:08
     - -------------------------------------- 0.4/12.8 MB 1.6 MB/s eta 0:00:08
     - -------------------------------------- 0.5/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.7/12.8 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/12.8 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/12.8 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.1/12.8 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.3/12.8 MB 2.5 MB/s eta 0:00:05
     ---- ----------------------------------- 1.4/12.8 MB 2

NER MODEL:
First we generate questions ->
Create the annotated dataset ->
Train the Model

In [ ]:
import json
import requests
import random

# Fetch book data from the URL
url = 'https://raw.githubusercontent.com/adibmenchali/bookworm/master/open_library_books.json'
response = requests.get(url)
books_data = response.json()

# Fetch questions from the URL
questions_url = 'https://raw.githubusercontent.com/adibmenchali/bookworm/master/questions.txt'
questions_response = requests.get(questions_url)
questions_lines = questions_response.text.strip().split('\n')

# Sample 10% of book titles
sampled_books = random.sample(books_data, k=int(0.10 * len(books_data)))

# Generate dataset
dataset = []

for book in sampled_books:
    title = book.get("title", "Unknown Title").lower()  # Convert title to lowercase so the model is no case sensitive
    for question_line in questions_lines:
        # Replace {title} with the book title
        question = question_line.format(title=title)
        # Create annotation for the question
        start_index = question.find(title)
        end_index = start_index + len(title)
        annotation = {"entities": [(start_index, end_index, "BOOK_TITLE")]}
        # Add annotated question and annotation to dataset
        dataset.append({"question": question, "annotation": annotation})

with open('annotated_dataset.json', 'w') as f:
    json.dump(dataset, f, indent=4)

print("Dataset generation complete.")

Dataset generation complete.


Train the model (takes a while to run that's why I saved it so I can just load it when needed)

In [1]:
import spacy
from spacy.training.example import Example
import json
from transformers import pipeline

# Load the BERT-based model from Hugging Face's Transformers library
nlp = pipeline("ner", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")

# Load the annotated dataset from a JSON file
def load_dataset_from_json(file_path):
    with open(file_path, 'r') as file:
        dataset = json.load(file)
    return dataset

# Convert annotations to SpaCy format
def convert_to_spacy_format(dataset):
    spacy_data = []
    for item in dataset:
        text = item['question'].lower()
        entities = item['annotation']['entities']
        spacy_entities = [(int(start), int(end), label) for start, end, label in entities]
        spacy_data.append((text, {"entities": spacy_entities}))
    return spacy_data

# Convert the annotated dataset to SpaCy format
def prepare_training_data(dataset):
    TRAIN_DATA = convert_to_spacy_format(dataset)
    return TRAIN_DATA

# Train the NER model
def train_ner_model(TRAIN_DATA):
    # Initialize a blank English model
    nlp_spacy = spacy.blank('en')

    # Add NER component to the pipeline
    ner = nlp_spacy.add_pipe('ner')

    # Add BOOK_TITLE as a new entity label
    ner.add_label('BOOK_TITLE')

    # Convert the SpaCy training data to Example objects
    examples = []
    for text, annotations in TRAIN_DATA:
        doc = nlp_spacy.make_doc(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)

    # Train the NER model
    nlp_spacy.begin_training()
    num_epochs = 10  # Number of training epochs
    print(f"Training the NER model for {num_epochs} epochs...")
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        losses = {}
        for example in examples:
            nlp_spacy.update([example], drop=0.5, losses=losses)  # Adjust dropout as needed
        print("Losses:", losses)

    return nlp_spacy

# Save the trained model
def save_model(nlp_spacy, output_dir):
    nlp_spacy.to_disk(output_dir)

# Main function
def main():
    # Load the annotated dataset
    dataset = load_dataset_from_json('annotated_dataset.json')

    # Prepare the training data
    TRAIN_DATA = prepare_training_data(dataset)

    # Train the NER model
    nlp_spacy = train_ner_model(TRAIN_DATA)

    # Save the trained model (optional)
    save_model(nlp_spacy, "custom_ner_model_with_bert")

    print("Custom NER model trained and saved with BERT embeddings.")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

This is an example test:

In [34]:
import spacy
# Load the trained SpaCy NER model
nlp = spacy.load("custom_ner_model_with_bert")

# Example text to test the NER model
text = "who is the author of the martian?"

# Process the text with the NER model
doc = nlp(text)

# Print the detected entities
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")


This is the chatbot integration, to make the code cleaner I put the question answering and recommendation model in different files which I imported here. I did the same thing for the NER model but I put it in this notebook to clarify the process...

In [4]:
import gradio as gr
import os
import time
from bookworm_ai import setup_indexing_pipeline, fetch_book_data, extract_book_title, create_document_objects, setup_qa_pipeline, get_best_answer
import spacy
import bookworm_recommender as rec


def print_like_dislike(x: gr.LikeData):
    print(x.index, x.value, x.liked)

def main(user_question):

    # Setup indexing pipeline
    indexing_pipeline, document_store = setup_indexing_pipeline()

    # Load the NER model
    nlp = spacy.load("custom_ner_model_with_bert")

    # Extract book title from user input
    book_title = extract_book_title(user_question, nlp)
    print("book title is", book_title)
    if book_title == user_question:
        # Call the recommendation system function
        book_df = rec.load_book_data('book_data.csv')
        print("book_df done")
        similar_books = rec.get_similar_books(book_df, book_title)
        formatted_books = "These are the recommended books:\n"
        for book in similar_books:

            formatted_books += f"{book['title']} by "
            formatted_books += "{}\n".format(book['author'].replace('[','').replace(']','').replace('\'',''))
            #formatted_books += f"Description: {book['description']}\n"
            #formatted_books += f"Similarity Score: {book['similarity_score']}\n\n"
        return formatted_books

    else:

        book_items = fetch_book_data(book_title)
        # Create document objects for each book item and aggregate descriptions
        documents = create_document_objects(book_items)

        if documents:
            indexing_pipeline.run({"documents": documents})
        else:
            print("Book data not found.")

        # Setup QA pipeline
        qa_pipeline = setup_qa_pipeline(document_store)

        # Get best answer
        best_answer = get_best_answer(qa_pipeline, user_question)

        return(best_answer)

def add_message(history, message):
    for x in message["files"]:
        history.append(((x,), None))
    if message["text"] is not None:
        history.append((message["text"], None))
    return history, gr.MultimodalTextbox(value=None, interactive=False)

def bot(history):
    user_input = history[-1][0]
    response = main(user_input)
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
    )

    chat_input = gr.MultimodalTextbox(interactive=True, file_types=["image"], placeholder="Ask a question about a book (To use the recommender system simply type a book title)...", show_label=False)

    chat_msg = chat_input.submit(add_message, [chatbot, chat_input], [chatbot, chat_input])
    bot_msg = chat_msg.then(bot, chatbot, chatbot, api_name="bot_response")
    bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, [chat_input])

    chatbot.like(print_like_dislike, None, None)

demo.queue()
demo.launch()

c:\Users\Adib\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adib\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Adib\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.25.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.


book title is the lonely polygamist


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ExtractedAnswer(query='who wrote the lonely polygamist?', score=0.7816624641418457, data='Brady Udall', document=Document(id=65ad1f0d548da496434f2ce036cb53c953522f6b6ba735b57a82e83ff61203f3, content: 'Title: The Lonely Polygamist;
Writer/Author: Brady Udall;
Topic: Golden Richards is a normal dad. Bu...', meta: {'title': 'The Lonely Polygamist', 'authors': 'Brady Udall', 'publisher': 'Random House', 'publishedDate': '2013-03-31', 'industryIdentifiers': "{'type': 'ISBN_13', 'identifier': '9781446477670'}, {'type': 'ISBN_10', 'identifier': '1446477673'}", 'readingModes': {'text': True, 'image': False}, 'pageCount': 608, 'printType': 'BOOK', 'categories': 'Fiction', 'maturityRating': 'NOT_MATURE', 'allowAnonLogging': True, 'contentVersion': '1.5.5.0.preview.2', 'panelizationSummary': {'containsEpubBubbles': False, 'containsImageBubbles': False}, 'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=j0FSHpXsaf0C&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api

## NER Evaluation

In [4]:
import pandas as pd

# Load the dataset into a pandas DataFrame
df = pd.read_csv('Best_Book_21st.csv')

In [5]:
df.set_index('id',inplace=True)

In [21]:
# Filter out rows with null values in the 'author' column
df_filtered = df.dropna(subset=['author'])

# Extract unique titles and authors from the filtered DataFrame
unique_titles_and_authors = df_filtered[['title', 'author']].drop_duplicates().iloc[:100].values.tolist()

unique_titles_and_authors


[['The Martian', 'Andy Weir'],
 ['Under the Banner of Heaven: A Story of Violent Faith', 'Jon Krakauer'],
 ['Cutting for Stone', 'Abraham   Verghese'],
 ['We Need to Talk About Kevin', 'Lionel Shriver'],
 ['The Immortal Life of Henrietta Lacks', 'Rebecca Skloot'],
 ['Eragon (The Inheritance Cycle, #1)', 'Christopher Paolini'],
 ['Unaccustomed Earth', 'Jhumpa Lahiri'],
 ['A Dance with Dragons (A Song of Ice and Fire, #5)', 'George R.R. Martin'],
 ['The Outlands  (The Outlands Saga #1)', 'Tyler Edwards'],
 ['Clockwork Angel (The Infernal Devices, #1)', 'Cassandra Clare'],
 ['World Without End (Kingsbridge, #2)', 'Ken Follett'],
 ['Animal, Vegetable, Miracle: A Year of Food Life',
  'Barbara Kingsolver,Steven L. Hopp,Camille Kingsolver,Richard A. Houser'],
 ['The Five People You Meet in Heaven', 'Mitch Albom'],
 ['On Beauty', 'Zadie Smith'],
 ['Shantaram', 'Gregory David Roberts'],
 ['Seabiscuit: An American Legend', 'Laura Hillenbrand'],
 ["The Titan's Curse (Percy Jackson and the Olympi

In [66]:
from bookworm_ai import setup_indexing_pipeline, fetch_book_data, extract_book_title, create_document_objects, setup_qa_pipeline, get_best_answer

book_titles = []
def main(user_question,ner):
    global book_titles
    # Setup indexing pipeline
    indexing_pipeline, document_store = setup_indexing_pipeline()

    # Load the NER model
    nlp = spacy.load(ner)

    # Extract book title from user input
    book_title = extract_book_title(user_question, nlp)
    book_titles.append(book_title)
    print(book_titles)
    print("book title is", book_title)

    book_items = fetch_book_data(book_title)
    # Create document objects for each book item and aggregate descriptions
    documents = create_document_objects(book_items)

    if documents:
        indexing_pipeline.run({"documents": documents})
    else:
        print("Book data not found.")

    # Setup QA pipeline
    qa_pipeline = setup_qa_pipeline(document_store)

    # Get best answer
    best_answer = get_best_answer(qa_pipeline, user_question)

    return(best_answer)


In [67]:
#THIS IS TO TEST THE STANDARD NER MODEL (used for evaluation only). This is not duplicated, I created this because the standard NER model
# doesn't recognize the book_title label.

def extract_entity(question, ner_model):
    global last_extracted_entity
    
    entities = ner_model(question)
    for ent in entities.ents:  # Iterate over recognized entities
        if ent.text.lower() == "it" and last_extracted_entity:
            return last_extracted_entity
        else:
            last_extracted_entity = ent.text
            return ent.text
    return None

book_titles = []
def main(user_question,ner):
    global book_titles
    # Setup indexing pipeline
    indexing_pipeline, document_store = setup_indexing_pipeline()

    # Load the NER model
    nlp = spacy.load(ner)

    # Extract book title from user input
    book_title = extract_entity(user_question, nlp)
    book_titles.append(book_title)
    print(book_titles)
    print("book title is", book_title)

    book_items = fetch_book_data(book_title)
    # Create document objects for each book item and aggregate descriptions
    documents = create_document_objects(book_items)

    if documents:
        indexing_pipeline.run({"documents": documents})
    else:
        print("Book data not found.")

    # Setup QA pipeline
    qa_pipeline = setup_qa_pipeline(document_store)

    # Get best answer
    best_answer = get_best_answer(qa_pipeline, user_question)

    return(best_answer)


The following code block runs the NER model and displays the predictions.
It's important to set the model to "en_core_web_sm" to test the standard NER model, otherwise "custom_ner_model_with_bert".

In [68]:
authors=[]
for title,author in unique_titles_and_authors:
    authors.append(author)

titles = []
for title, author in unique_titles_and_authors:
    titles.append(title)

questions = []

for title in titles:
    questions.append(f"Who is the author of {title.lower()}?")

#IMPORTANT: set the model to "en_core_web_sm" to test the standard NER model, otherwise "custom_ner_model_with_bert".
predictions = []
for question in questions:
    predictions.append(main(question,"custom_ner_model_with_bert"))

['martian']
book title is martian


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ExtractedAnswer(query='Who is the author of the martian?', score=0.7729896306991577, data='Andy Weir', document=Document(id=752f22e6ddd48180b86903ed2c2243922f8dea1660e18807cb436b90c87f1c33, content: 'Title: Sopravvissuto - The Martian;
Writer/Author: Andy Weir;
Topic: Da questo romanzo il grande fil...', meta: {'title': 'Sopravvissuto - The Martian', 'authors': 'Andy Weir', 'publisher': 'Newton Compton Editori', 'publishedDate': '2014-10-23', 'industryIdentifiers': "{'type': 'ISBN_13', 'identifier': '9788854170520'}, {'type': 'ISBN_10', 'identifier': '8854170526'}", 'readingModes': {'text': True, 'image': True}, 'pageCount': 384, 'printType': 'BOOK', 'categories': 'Fiction', 'averageRating': 5, 'ratingsCount': 3, 'maturityRating': 'NOT_MATURE', 'allowAnonLogging': True, 'contentVersion': '1.11.13.0.preview.3', 'panelizationSummary': {'containsEpubBubbles': False, 'containsImageBubbles': False}, 'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=XWMGBAAAQBAJ&prin

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ExtractedAnswer(query='Who is the author of clockwork prince (the infernal devices, #2)?', score=0.6311920285224915, data='Angelo Pezzana', document=Document(id=01cdab9394850832927c46cc52f8160140eb174ce94b3c6b714e0705fa7c6b14, content: 'Title: Storia della citta di Parma continuata;
Writer/Author: Angelo Pezzana;
Topic: Sorry, I am una...', meta: {'title': 'Storia della citta di Parma continuata', 'subtitle': '1401 - 1449. 2', 'authors': 'Angelo Pezzana', 'publishedDate': '1842', 'industryIdentifiers': "{'type': 'OTHER', 'identifier': 'ONB:+Z168004004'}", 'readingModes': {'text': False, 'image': True}, 'pageCount': 882, 'printType': 'BOOK', 'maturityRating': 'NOT_MATURE', 'allowAnonLogging': False, 'contentVersion': '0.6.5.0.full.1', 'panelizationSummary': {'containsEpubBubbles': False, 'containsImageBubbles': False}, 'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=pc9TAAAAcAAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api', 'thumbnail': 'http://b

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ExtractedAnswer(query='Who is the author of saturday?', score=0.820523738861084, data='Robert Burns', document=Document(id=f8db19f23fbeb1ad7b58637364d19d1fb51cab38babfd6bb55869079ca50e977, content: 'Title: The Cotter's Saturday Night;
Writer/Author: Robert Burns;
Topic: Sorry, I am unable to answer...', meta: {'title': "The Cotter's Saturday Night", 'authors': 'Robert Burns', 'publishedDate': '1803', 'industryIdentifiers': "{'type': 'OTHER', 'identifier': 'NLS:B900060635'}", 'readingModes': {'text': False, 'image': True}, 'pageCount': 20, 'printType': 'BOOK', 'maturityRating': 'NOT_MATURE', 'allowAnonLogging': False, 'contentVersion': '0.1.2.0.full.1', 'panelizationSummary': {'containsEpubBubbles': False, 'containsImageBubbles': False}, 'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=LsRp-Iool2IC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api', 'thumbnail': 'http://books.google.com/books/content?id=LsRp-Iool2IC&printsec=frontcover&img=1&zoom=1&edge

In [63]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from haystack import Pipeline
from haystack.components.evaluators import SASEvaluator

def ner_eval_pipeline(titles, book_titles):
    # Flatten the lists and join tokens within each sublist
    y_true = [''.join(sublist).lower() for sublist in titles]
    print(y_true)
    y_pred = [''.join(sublist) for sublist in book_titles]

    eval_pipeline = Pipeline()
    sas_evaluator = SASEvaluator()

    eval_pipeline.add_component("sas_evaluator", sas_evaluator)
    result = eval_pipeline.run(
        {
            "sas_evaluator": {
                "ground_truth_answers": titles,
                "predicted_answers": book_titles
            }
        }
    )

    # Print individual scores and overall score from SAS Evaluator
    for evaluator in result:
        print(result[evaluator]["individual_scores"])

    for evaluator in result:
        print(result[evaluator]["score"])

    # Calculate and print standard performance metrics
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred, average='macro'))
    print("Recall: ", recall_score(y_true, y_pred, average='macro'))
    print("F1 Score: ", f1_score(y_true, y_pred, average='macro'))
    
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    
    return result


In [71]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from haystack import Pipeline
from haystack.components.evaluators import SASEvaluator

def standard_ner_eval_pipeline(titles, book_titles):
    # Filter out None values from predicted answers and corresponding entries from ground truth
    filtered_titles = []
    filtered_book_titles = []
    for title, book_title in zip(titles, book_titles):
        if book_title is not None:
            filtered_titles.append(title)
            filtered_book_titles.append(book_title)

    # Flatten the filtered lists and join tokens within each sublist
    y_true = [''.join(sublist).lower() for sublist in filtered_titles]
    y_pred = [''.join(sublist) for sublist in filtered_book_titles]

    eval_pipeline = Pipeline()
    sas_evaluator = SASEvaluator()

    eval_pipeline.add_component("sas_evaluator", sas_evaluator)
    result = eval_pipeline.run(
        {
            "sas_evaluator": {
                "ground_truth_answers": filtered_titles,
                "predicted_answers": filtered_book_titles
            }
        }
    )

    # Print individual scores and overall score from SAS Evaluator
    for evaluator in result:
        print(result[evaluator]["individual_scores"])

    for evaluator in result:
        print(result[evaluator]["score"])

    # Calculate and print standard performance metrics
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred, average='macro'))
    print("Recall: ", recall_score(y_true, y_pred, average='macro'))
    print("F1 Score: ", f1_score(y_true, y_pred, average='macro'))
    
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    
    return result


In [72]:
result_standard_ner = standard_ner_eval_pipeline(titles,book_titles)

[0.8962118625640869, 0.7029730081558228, 0.6668420433998108, 0.36934199929237366, 0.45133787393569946, 0.3472628593444824, 0.4344772398471832, 0.5271625518798828, 0.33469951152801514, 0.4629645347595215, 0.4121271073818207, 0.5116442441940308, 1.0, 0.37526556849479675, 0.5676705837249756, 0.8062781095504761, 0.375021755695343, 0.5266337394714355, 0.2887517213821411, 0.6829578876495361, 0.6377651691436768, 0.6947778463363647, 0.8427202105522156, 0.42410367727279663, 0.4340893030166626, 0.4038134217262268, 0.38636699318885803, 0.4247820973396301, 0.9086362719535828, 0.4459054172039032, 0.4057459831237793, 0.665999710559845, 0.2889268398284912, 0.6377964019775391, 0.7100322842597961, 0.4482833445072174, 0.3545823395252228, 0.5833338499069214, 0.833564817905426, 0.23471270501613617, 0.4767560362815857, 0.42838698625564575, 0.5907237529754639, 0.5314612984657288, 0.4397408664226532, 0.39759430289268494, 0.43822333216667175, 0.5637582540512085, 0.5222235918045044, 0.976972222328186, 0.445371

c:\Users\Adib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Adib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Adib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{

In [64]:
result_ner = ner_eval_pipeline(titles,book_titles)

['the martian', 'under the banner of heaven: a story of violent faith', 'cutting for stone', 'we need to talk about kevin', 'the immortal life of henrietta lacks', 'eragon (the inheritance cycle, #1)', 'unaccustomed earth', 'a dance with dragons (a song of ice and fire, #5)', 'the outlands  (the outlands saga #1)', 'clockwork angel (the infernal devices, #1)', 'world without end (kingsbridge, #2)', 'animal, vegetable, miracle: a year of food life', 'the five people you meet in heaven', 'on beauty', 'shantaram', 'seabiscuit: an american legend', "the titan's curse (percy jackson and the olympians, #3)", 'shadow kiss (vampire academy, #3)', 'the battle of the labyrinth (percy jackson and the olympians, #4)', '2666', 'frostbite (vampire academy, #2)', 'the lost symbol (robert langdon, #3)', 'olive kitteridge (olive kitteridge, #1)', 'the last lecture', "miss peregrine's home for peculiar children (miss peregrine's peculiar children, #1)", 'me before you (me before you, #1)', 'the tales of

c:\Users\Adib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Adib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Adib\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{

## QA Evaluation

In [ ]:
from haystack import Pipeline
from haystack.components.evaluators import SASEvaluator

def setup_eval_pipeline(authors, predictions):

    eval_pipeline = Pipeline()
    sas_evaluator = SASEvaluator()

    eval_pipeline.add_component("sas_evaluator", sas_evaluator)
    result = eval_pipeline.run(
            {
            "sas_evaluator": {"ground_truth_answers": authors,
            "predicted_answers": predictions}
        }
    )

    for evaluator in result:
        print(result[evaluator]["individual_scores"])

    for evaluator in result:
        print(result[evaluator]["score"])


    return result

In [ ]:
result = setup_eval_pipeline(authors,predictions)

The evaluation of the the recommendation systems is conducted in the notebook bookworm_rec_eval.ipynb.